# Gensim 패키지
- Python으로 작성된 오픈 소스 라이브러리로, 자연어 처리와 관련된 다양한 기능을 제공한다.
- 주요 기능
    - **Word Embeddings**
        - word2vec, fastext, doc2vec 등 다양한 word embedding 모델을 제공
    - **토픽 모델링 (Topic Modeling)**
        - LDA등 문장의 주제를 파악하는 모델 제공
    - **텍스트/word 유사도 계산**
    - **문서 군집화**
        - 비슷한 주제의 문서들을 군집화.
    - 다양한 dataset과 pretrained model 제공
        - https://github.com/piskvorky/gensim-data
- https://radimrehurek.com/gensim/

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 17.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 71.1 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


## 설치
- `pip install gensim`

# Word2Vec 학습

- gensim.models.Word2Vec
- 주요 파라미터
    - sentences
        -  학습에 사용할 문장의 리스트. 각 문장의 단어들을 리스트는다. 문장들을 리스트로 묶어 입력한다.
        - 예시: \[\['word1', 'word2', 'word3'], \['word4', 'word5']]
    - vector_size
        -  embedding vector 크기. 기본값: 100
    - window
        -  context window 크기. 중심단어를 기준으로 좌우 몇개의 단어를 확인하는지 크기. 기본값: 5
    - min_count
        - 이 설정보다 낮은 빈도로 등장하는 단어는 무시한다. 데이터 노이즈를 줄이는데 도움이된다. 기본값: 5
    - sg
        - 모델 아키텍처 결정.
        - `0`: CBOW, `1`: Skip-gram. 기본값: 0
    - epochs
        - epochs 수 설정. 기본값: 5
    - alpha
        - initial leaning rate. 기본값: 0.025
    - min_alpha
        - 최소 learning rate. 기본값: 0.0001
        - epoch 마다 learning rate를 alpha 에서 min_alpha 까지 선형적으로 줄여나간다.
    - workers
        -  사용 Thread 수. 기본값: 3

## 학습(Train)
1. Word2Vec 의 initializer에 sentences를 넣어 한번에 학습한다.
2. Word2Vec 클래스에 학습 설정을 하고 `train()` 메소드를 이용해 학습한다.
    - epoch 단위로 작업을 할 경우 사용

In [2]:
# 샘플 텍스트 데이터
sentences = [
    "Natural language processing is an exciting field of study",
    "Word embeddings are a type of word representation",
    "Gensim is a powerful library for text processing",
    "Word2Vec creates vector representations of words", 
    "Gensim runs on Linux, Windows and OS X, as well as any other platform that supports Python and NumPy."
    "All Gensim source code is hosted on Github under the GNU LGPL license, maintained by its open source community.",
    "For commercial arrangements, see Business Support.",
    "Gensim can process arbitrarily large corpora, using data-streamed algorithms.",
    "There are no \"dataset must fit in RAM\" limitations."
]

In [26]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import os
import re

def tokenizer(docs) :
    # 모두 소문자로 변경, 소문자 숫자 underscore와 공백 문자 제외한 나머지 제거
    sent = [word_tokenize(re.sub(r'[^\w\s]',"",sentence.lower())) for sentence in docs]
    # 단어 단위 토큰화
    return sent

tokens = tokenizer(sentences)

In [27]:
# 학습 - 객체 생성하면서 학습 데이터를 넣어주면 학습 데이터 반환
model1 = Word2Vec(
    sentences=tokens, #학습데이터
    vector_size=10, # embedding vector 차원
    window=2, # 중심단어 기준 몇 개 정할지
    min_count=1,  # 최소 출연 빈도 수, 지정한 빈도수보다 적게나오면 Vocab에 넣지않음
    epochs=10,
    workers=os.cpu_count()  # 병렬 처리 시 사용할 processor 
)

In [28]:
# 학습2 -> train() 사용
# 학습 관련 설정만 setting
model2 = Word2Vec(
    vector_size=10,
    window=2,
    min_count=1,
    workers=os.cpu_count()
)
# 학습 데이터 넣어서 사전(vocab)을 모델에 생성
model2.build_vocab(tokens)

# 학습처리
epochs = 10
for epoch in range(epochs) :
    model2.train(
        tokens,  # 학습 데이터 
        total_examples=model2.corpus_count,  # 학습 데이터 갯수
        epochs=1,
        compute_loss=True  # 학습이 끝나면 loss 계산
    )
    # loss 조회
    loss = model2.get_latest_training_loss() #compute_loss = True이면 학습 끝나고 계산

    print(f'{epoch+1} 에폭의 loss는 {loss}이다')

1 에폭의 loss는 147.74391174316406이다
2 에폭의 loss는 139.40313720703125이다
3 에폭의 loss는 108.19081115722656이다
4 에폭의 loss는 159.9152069091797이다
5 에폭의 loss는 163.42904663085938이다
6 에폭의 loss는 143.5673370361328이다
7 에폭의 loss는 136.47251892089844이다
8 에폭의 loss는 130.15609741210938이다
9 에폭의 loss는 164.7725067138672이다
10 에폭의 loss는 156.24295043945312이다


## 학습 후 결과 조회

- **KeyedVectors 조회**
    - KeyedVectors는 단어와 vector를 매핑한 객체로 embedding vector를 이용한 다양한 조회를 지원한다.
    - model.wv 로 조회해서 사용.
- **Embedding Vector 조회**
  - model.wv.vectors
- **단어 목록 조회**
    - model.wv.index_to_key, model.wv.key_to_index
- **단어 벡터 조회**
    - model.wv[word]: 특정 단어의 vector반환
- **Vocab에 대상 단어가 있는지 확인**
    - model.wv.\_\_contains\_\_(word)
- **유사단어들 찾기**
    - model.wv.most_similar(word)
- **단어간 유사도 비교**
    - model.wv.similarity(word1, word2)
- 유사도를 계산할 때 **코사인 유사도(Cosine Similarity)** 를 사용한다.

In [13]:
model1.wv

In [14]:
model1.wv.index_to_key

['gensim',
 'is',
 'of',
 'for',
 'processing',
 'source',
 'as',
 'on',
 'word',
 'are',
 'a',
 'and',
 'words',
 'runs',
 'linux',
 'os',
 'windows',
 'vector',
 'x',
 'well',
 'any',
 'representations',
 'limitations',
 'creates',
 'word2vec',
 'text',
 'platform',
 'library',
 'powerful',
 'representation',
 'type',
 'embeddings',
 'study',
 'field',
 'exciting',
 'an',
 'language',
 'other',
 'that',
 'ram',
 'see',
 'support',
 'can',
 'process',
 'arbitrarily',
 'large',
 'corpora',
 'using',
 'datastreamed',
 'algorithms',
 'there',
 'no',
 'dataset',
 'must',
 'fit',
 'in',
 'business',
 'arrangements',
 'supports',
 'commercial',
 'python',
 'numpyall',
 'code',
 'hosted',
 'github',
 'under',
 'the',
 'gnu',
 'lgpl',
 'license',
 'maintained',
 'by',
 'its',
 'open',
 'community',
 'natural']

In [15]:
model1.wv.key_to_index

{'gensim': 0,
 'is': 1,
 'of': 2,
 'for': 3,
 'processing': 4,
 'source': 5,
 'as': 6,
 'on': 7,
 'word': 8,
 'are': 9,
 'a': 10,
 'and': 11,
 'words': 12,
 'runs': 13,
 'linux': 14,
 'os': 15,
 'windows': 16,
 'vector': 17,
 'x': 18,
 'well': 19,
 'any': 20,
 'representations': 21,
 'limitations': 22,
 'creates': 23,
 'word2vec': 24,
 'text': 25,
 'platform': 26,
 'library': 27,
 'powerful': 28,
 'representation': 29,
 'type': 30,
 'embeddings': 31,
 'study': 32,
 'field': 33,
 'exciting': 34,
 'an': 35,
 'language': 36,
 'other': 37,
 'that': 38,
 'ram': 39,
 'see': 40,
 'support': 41,
 'can': 42,
 'process': 43,
 'arbitrarily': 44,
 'large': 45,
 'corpora': 46,
 'using': 47,
 'datastreamed': 48,
 'algorithms': 49,
 'there': 50,
 'no': 51,
 'dataset': 52,
 'must': 53,
 'fit': 54,
 'in': 55,
 'business': 56,
 'arrangements': 57,
 'supports': 58,
 'commercial': 59,
 'python': 60,
 'numpyall': 61,
 'code': 62,
 'hosted': 63,
 'github': 64,
 'under': 65,
 'the': 66,
 'gnu': 67,
 'lgpl': 

In [29]:
# 특정 단어의 embedding vector 조회
model1.wv['gensim']

array([-0.00530078,  0.00215998,  0.0511257 ,  0.08994552, -0.0929675 ,
       -0.07124555,  0.06510045,  0.08978292, -0.05092729, -0.0380263 ],
      dtype=float32)

In [30]:
# vocab에 단어 있는지 확인
model1.wv.__contains__('gensim'), model1.wv.__contains__('.')

(True, False)

In [31]:
word = '.'

if model1.wv.__contains__(word) :
    print(model1.wv[word])
else :
    print(word,'는 없는 단어')

. 는 없는 단어


> # 코사인 유사도
> - 두 벡터 간의 유사성을 측정하는 중요한 방법 중 하나.
> - 코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 유사도를 계산한다. 이때 벡터의 크기는 결과에 영향을 미치지 않고, 오직 방향만이 중요하다.
> ## 공식
> 
> $$ similarity = cos(\theta) = \frac{A⋅B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n}{A_i×B_i}}{\sqrt{\sum_{i=1}^{n}(A_i)^2}×\sqrt{\sum_{i=1}^{n}(B_i)^2}} $$
> 
> ## 결과 해석
> 
> - **값의 범위**: -1에서 1 사이의 값을 가집니다
>   - 1: 두 벡터가 완전히 동일한 방향 (0도의 cosine 값)
>   - 0: 두 벡터가 직교 (90도의 cosine 값)
>   - -1: 두 벡터가 정반대 방향 (180도의 cosine 값)
> 
> ![cosine_similarity](figures/gensim_consin_sim.png)
>
> ## Python 코사인 유사도 계산
> ```python
> from numpy import dot
> from numpy.linalg import norm
> 
> def cosine_similarity(A, B):
>     return dot(A, B)/(norm(A)*norm(B))
> ```



In [32]:
# 유사도 관련 메소드
# 유사도 계산 - 코사인 유사도
model1.wv.most_similar('gensim', topn=3) # 조회 단어와 유사도가 높은 10개 단어 반환


[('numpyall', 0.7189716696739197),
 ('study', 0.7138967514038086),
 ('that', 0.6710396409034729)]

In [33]:
# 단어 간의 유사 계산
model1.wv.similarity('gensim', 'processing')

-0.22607413

In [25]:
model1.wv.vectors # 모든 embedding된 vector

array([[-5.30078216e-03,  2.15998176e-03,  5.11256978e-02,
         8.99455249e-02, -9.29674953e-02, -7.12455511e-02,
         6.51004538e-02,  8.97829160e-02, -5.09272851e-02,
        -3.80263031e-02],
       [ 7.39076957e-02, -1.54466406e-02, -4.53684442e-02,
         6.55177534e-02, -4.86253649e-02, -1.80424210e-02,
         2.90737022e-02,  9.82502103e-03, -8.31743106e-02,
        -9.47242603e-02],
       [ 7.32511505e-02,  5.05392328e-02,  6.75722361e-02,
         7.46826502e-03,  6.35220706e-02, -3.40774544e-02,
        -9.01477318e-03,  5.78403622e-02, -7.59721324e-02,
        -3.98136228e-02],
       [-7.50671700e-02, -9.38217156e-03,  9.54507515e-02,
        -7.31930435e-02, -2.33855415e-02, -1.93917714e-02,
         8.09578523e-02, -5.92647381e-02,  1.99908551e-04,
        -4.77067195e-02],
       [-9.59166065e-02,  5.00562377e-02, -8.76059458e-02,
        -4.38948721e-02, -3.71427013e-04, -2.90183281e-03,
        -7.63971284e-02,  9.61731896e-02,  4.95607182e-02,
         9.

## 모델 저장 및 로딩

### 모델 저장, 로딩
- `model.save('저장파일 경로')`
  - gensim 자체 포맷으로 저장된다.
- `gensim.models.Word2Vec.load('저장파일 경로')`
  - `model.save()`로 저장된 모델을 Loading한다.

In [ ]:
os.makedirs('saved_model/word2vec', exist_ok=True)
model1.save('saved_model/word2vec/sample_w2v.model')

In [36]:
loadmodel = Word2Vec.load('saved_model/word2vec/sample_w2v.model')
loadmodel.wv.most_similar('gensim')

[('numpyall', 0.7189716696739197),
 ('study', 0.7138967514038086),
 ('that', 0.6710396409034729),
 ('can', 0.6415793299674988),
 ('datastreamed', 0.5988761186599731),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.507987380027771)]

### Word Embedding Vector만 저장 및 로드
- `KeyedVectors` 를 이용해 저장한다.
    - `model.wv.save_word2vec_format('저장경로', binary=True|False)`
        - binary=True: binary 파일로 저장한다. 용량이 작은 대신 내용확인이 안된다.
        - binary=False: csv(공백구분자) 형식 text로 저장한다. 내용을 확인할 수있지만 파일용량이 크다.
- `KeyedVectors.load_word2vec_format("저장경로", binary=True|False)`
    - 저장시 binary에 맞춰 읽는다.

In [47]:
model1.wv.save_word2vec_format('saved_model/word2vec/sample_w2v.model.bin', binary=True)
model1.wv.save_word2vec_format('saved_model/word2vec/sample_w2v.model.csv', binary=False)

In [48]:
from gensim.models import KeyedVectors
load_wv1 = KeyedVectors.load_word2vec_format('saved_model/word2vec/sample_w2v.model.bin', binary=True)
load_wv2 = KeyedVectors.load_word2vec_format('saved_model/word2vec/sample_w2v.model.csv', binary=False)

load_wv1.most_similar('gensim')
load_wv2.most_similar('gensim')

[('numpyall', 0.7189716696739197),
 ('study', 0.7138967514038086),
 ('that', 0.6710396409034729),
 ('can', 0.6415793299674988),
 ('datastreamed', 0.5988761186599731),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.507987380027771)]

## Pretrained 모델 사용하기
- https://github.com/Kyubyong/wordvectors

In [49]:
from zipfile import ZipFile

with ZipFile('ko_new.zip') as zf :
    zf.extractall('saved_model/word2vec')

In [51]:
ko_wv = KeyedVectors.load_word2vec_format(
    "saved_model/word2vec/ko_new.txt",
    binary=False
)

In [55]:
word = '컴퓨터'
vec = ko_wv[word]
print(ko_wv.most_similar(word, topn=5))

[('하드웨어', 0.7727355360984802), ('소프트웨어', 0.730889618396759), ('마이크로프로세서', 0.7205492258071899), ('개인용', 0.7154534459114075), ('노트북', 0.7130324244499207)]


# 빅카인즈 뉴스 데이터를 이용한 Word2Vec 학습
- 빅카인즈
    - 한국언론진흥재단에서 운영하는 뉴스빅데이터 분석 서비스 사이트
- 빅카인즈에서 특정 분야의 기사들을 수집해서 학습시킨다.
    - https://www.bigkinds.or.kr/
    - 회원가입 (구글, 네이버, 카카오 계정으로 가입 가능) 후 로그인
    - 뉴스분석 > 뉴스검색$\cdot$분석 클릭
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds1.png)
    - 기간, 언론사, 분류, 상세검색 등 검색 조건입력 후 조회
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds2.png)
    - 결과 다운로드
        - step3 분석결과및 시각화 -> 맨 아래 `엑셀다운로드` 클릭 
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds3.png)

In [57]:
!pip install openpyxl

In [58]:
import pandas as pd
df = pd.read_excel('dataset/NewsResult_20240520-20241120.xlsx')

/opt/miniconda3/envs/ml/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df = df[['제목', '본문']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      20000 non-null  object
 1   본문      20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


In [63]:
# 전처리 + 토큰화
from konlpy.tag import Okt

okt = Okt()

def tokenizer(text) :
    text = re.sub(r'[^\w\s]', '', text)  # 글자와 공백만 남겨둠
    # 2글자 이상인 형용사, 명사, 동사만 추출
    tokens = [word for word, pos in okt.pos(text) 
              if pos in ['Adjective', 'Noun', 'Verb'] and len(word) > 1]
    return tokens

tokenizer('2글자 이상인 형용사, 명사, 동사만 추출')


['글자', '이상인', '형용사', '명사', '동사', '추출']

In [64]:
# dataset 생성
token_list = []
for idx, row in df.iterrows() :  # tuple return (행 idx, 행 데이터 - Series)
    row_token = tokenizer(row['제목'] + ' ' + row['본문'])
    token_list.append(row_token)

token_list

[['에스테',
  '둔산',
  '리프',
  '센트럴',
  '순위',
  '청약',
  '에스테',
  '시행',
  '계룡',
  '건설',
  '시공',
  '맡은',
  '둔산',
  '리프',
  '센트럴',
  '순위',
  '청약',
  '접수',
  '진행',
  '한다',
  '청약',
  '접수',
  '한국',
  '부동산',
  '청약',
  '오전',
  '오후',
  '사이',
  '진행',
  '된다',
  '당첨',
  '발표',
  '정당',
  '계약',
  '사흘',
  '진행',
  '된다',
  '청약',
  '대전광역시',
  '세종',
  '특별자치시',
  '충청남도',
  '거주',
  '하는'],
 ['한국',
  '남부',
  '발전',
  '하동',
  '드림',
  '본부',
  '유류',
  '화재',
  '복합',
  '재난',
  '대응',
  '훈련',
  '실시',
  '사진',
  '한국',
  '남부',
  '발전',
  '한국',
  '남부',
  '발전',
  '하동',
  '드림',
  '본부',
  '유관',
  '기관',
  '합동',
  '대규모',
  '재난대응',
  '훈련',
  '하동',
  '소방',
  '해양경찰',
  '해양',
  '환경',
  '공단',
  '하동',
  '수협',
  '기관',
  '참여',
  '가운데',
  '시행',
  '했다고',
  '밝혔다',
  '훈련',
  '방제',
  '소방차',
  '구급차',
  '공차',
  '유회',
  '수기',
  '드론',
  '재난',
  '관리자',
  '동원',
  '됐다',
  '해상'],
 ['둔산',
  '리프',
  '센트럴',
  '순위',
  '청약',
  '접수',
  '에스테',
  '계룡',
  '건설',
  '공급',
  '하는',
  '둔산',
  '리프',
  '센트럴',
  '오늘',
  '순위',
  '청약',
  '진행',
  '한다',
  '청약',
  '일정',
  '

In [66]:
from gensim.models import Word2Vec

model = Word2Vec(
    vector_size=100,  # embedding vector 차원 수
    window=5,
    sg=1, # 0 : CBOW / 1 : Skip-gram
    workers=os.cpu_count()
)

#Vocab 생성
model.build_vocab(token_list)
loss_list = []
best_score = float('inf')
save_path = 'saved_model/word2vec/bigkinds.model'

for epoch in range(1000) :
    model.train(
        token_list,
        total_examples=model.corpus_count,
        epochs=1,
        compute_loss=True
    )
    loss = model.get_latest_training_loss()
    if best_score > loss :
        model.save(save_path)
        print('모델 저장')
        best_score = loss
    loss_list.append(loss)
    if epoch % 50 == 0 or (epoch+1) == 1000 :
        print(f'[{epoch}/1000] loss : {loss}')

모델 저장
[0/1000] loss : 1232621.625
모델 저장
모델 저장
모델 저장
모델 저장
모델 저장
모델 저장
모델 저장
모델 저장
[50/1000] loss : 782795.875
모델 저장
모델 저장
모델 저장
모델 저장
[100/1000] loss : 679023.4375
모델 저장
[150/1000] loss : 765437.5625
[200/1000] loss : 685899.0
모델 저장
[250/1000] loss : 762393.375
모델 저장
모델 저장
[300/1000] loss : 687260.4375
모델 저장
[350/1000] loss : 678701.0
모델 저장
[400/1000] loss : 758658.6875
[450/1000] loss : 674638.625
[500/1000] loss : 670759.5625
[550/1000] loss : 755257.5625
[600/1000] loss : 754415.125
[650/1000] loss : 751561.25
[700/1000] loss : 688953.4375
[750/1000] loss : 754122.9375
[800/1000] loss : 669088.0625
[850/1000] loss : 755492.5
[900/1000] loss : 751567.625
[950/1000] loss : 755649.625
[999/1000] loss : 758446.1875


In [ ]:
load_model = Word2Vec.load(save_path)

#Vocab 크기
wv = load_model.wv
len(wv.key_to_index)

13464

In [73]:
# 유사도 조회
word = '폭염'
try :
    result = wv.most_similar(word)
    print(result)
except :
    print('없는 단어')

[('무더위', 0.7932885885238647), ('연일', 0.715847373008728), ('지속', 0.6852232813835144), ('더위', 0.6808524131774902), ('특보', 0.6658097505569458), ('계속', 0.659516453742981), ('질환', 0.6271578669548035), ('내려진', 0.6211817860603333), ('열대야', 0.6056889295578003), ('보가', 0.5939865708351135)]
